<a href="https://colab.research.google.com/github/Dark-Sied/Intent_Classification/blob/master/Intent_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [2]:
def load_dataset(filename):
#   df = pd.read_csv(filename, encoding = "latin1", names = ["Sentence", "Intent"])
    df = pd.read_excel(filename, sheet_name="dataset_XY", encoding="utf8")
    Label_tag = pd.read_excel(filename, sheet_name="Label_tag", encoding="utf8")  
    print(df.head())
    intent = df['Label']
    for idx, i in enumerate(intent):
#         print(i)
        intent[idx] = Label_tag.loc[i,"Tag"]
#     print("intent: ", intent)
    unique_intent = list(set(intent))
    sentences = list(df["Questions"])
  
    return (intent, unique_intent, sentences)


In [3]:
intent, unique_intent, sentences = load_dataset("dataset_XY_XLS_updatedbytho.xls")
print("Unique intents: ", unique_intent)
print("Number of intents: ", len(unique_intent))
# print(intent[:5])
# len(intent)

           Questions  Label
0               Chào      0
1    Dạo này thế nào      0
2  Có ai ở đây không      0
3           Xin chào      0
4           Xao chìn      0


C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Unique intents:  ['Tạm_biệt', 'Chào_hỏi', 'Nơi_nộp_hồ_sơ', 'Thời_gian_trả_kết_quả', 'Trình_tự ', 'Đối_tượng_xin_cấp_hộ_chiếu', 'Lệ_phí', 'Thời_gian_giải_quyết_hộ_chiếu', 'Nơi_nhận_hộ_chiếu', 'Chuẩn_bị_hồ_sơ', 'Chức_năng_của_bot', 'Cảm_ơn', 'Tên_bot', 'Thời_gian_tiếp_nhận_hồ_sơ', 'Các_loại_hộ_chiếu']
Number of intents:  15


In [4]:
# Hàm được viết bởi thọ
import pyvi.ViTokenizer as viToken
def viTokenList(word):
    return viToken.tokenize(word).split()
viTokenList("Anh thọ quá đẹp trai!")

['Anh', 'thọ', 'quá', 'đẹp_trai', '!']

In [5]:
print(sentences[:5])

['Chào', 'Dạo này thế nào', 'Có ai ở đây không', 'Xin chào', 'Xao chìn']


In [6]:
# nltk.download("stopwords")
# nltk.download("punkt")

In [7]:
#define stemmer
# stemmer = LancasterStemmer()

In [8]:
def cleaning(sentences):
    words = []
    for s in sentences:
#         clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s) # Find and replace duplicate 
        w = viTokenList(s)
        #stemming
        words.append([i.lower() for i in w])

    return words  

In [9]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
  


750
[['chào'], ['dạo', 'này', 'thế_nào']]


In [10]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
    token = Tokenizer(filters = filters)
    token.fit_on_texts(words)
    return token

In [11]:
def max_length(words):
    return(len(max(words, key = len)))
  

In [12]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 318 and Maximum length = 16


In [13]:
def encoding_doc(token, words):
    return(token.texts_to_sequences(words))

In [14]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [15]:
def padding_doc(encoded_doc, max_length):
    return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [16]:
padded_doc = padding_doc(encoded_doc, max_length)

In [17]:
padded_doc[:5]

array([[115,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [180,  99,  21,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  5,  55,  24, 126,   3,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [ 17, 115,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [204,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]])

In [18]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (750, 16)


In [19]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')


In [20]:
output_tokenizer.word_index

{'tạm_biệt': 1,
 'chào_hỏi': 2,
 'nơi_nộp_hồ_sơ': 3,
 'thời_gian_trả_kết_quả': 4,
 'trình_tự': 5,
 'đối_tượng_xin_cấp_hộ_chiếu': 6,
 'lệ_phí': 7,
 'thời_gian_giải_quyết_hộ_chiếu': 8,
 'nơi_nhận_hộ_chiếu': 9,
 'chuẩn_bị_hồ_sơ': 10,
 'chức_năng_của_bot': 11,
 'cảm_ơn': 12,
 'tên_bot': 13,
 'thời_gian_tiếp_nhận_hồ_sơ': 14,
 'các_loại_hộ_chiếu': 15}

In [21]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [22]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [23]:
encoded_output.shape

(750, 1)

In [24]:
def one_hot(encode):
    o = OneHotEncoder(sparse = False)
    return(o.fit_transform(encode))

In [25]:
output_one_hot = one_hot(encoded_output)

C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [26]:
output_one_hot.shape

(750, 15)

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
# print(len(padded_doc))
# print(len(output_one_hot))
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)
train_X, batch_X, train_Y, batch_Y = train_test_split(train_X, train_Y, shuffle = True, test_size = 0.2)

In [30]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))
print("Shape of batch_X = %s and batch_Y = %s" % (batch_X.shape, batch_Y.shape))

Shape of train_X = (480, 16) and train_Y = (480, 15)
Shape of val_X = (150, 16) and val_Y = (150, 15)
Shape of batch_X = (120, 16) and batch_Y = (120, 15)


In [31]:
def create_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
    model.add(Bidirectional(LSTM(128)))
    #   model.add(LSTM(128))
    model.add(Dense(32, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(15, activation = "softmax"))

    return model

In [32]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 16, 128)           40704     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 15)                495       
Total params: 312,591
Trainable params: 271,887
Non-trainable params: 40,704
_________________________________________________________________


In [42]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 300, batch_size = 32, validation_data = (batch_X, batch_Y), callbacks = [checkpoint])

Train on 480 samples, validate on 120 samples
Epoch 1/300
480/480 [==============================] - ETA: 36s - loss: 0.3438 - acc: 0.93 - ETA: 17s - loss: 0.4871 - acc: 0.82 - ETA: 11s - loss: 0.4801 - acc: 0.80 - ETA: 8s - loss: 0.5024 - acc: 0.7891 - ETA: 6s - loss: 0.5026 - acc: 0.806 - ETA: 4s - loss: 0.4745 - acc: 0.817 - ETA: 3s - loss: 0.4823 - acc: 0.812 - ETA: 2s - loss: 0.4993 - acc: 0.812 - ETA: 2s - loss: 0.4867 - acc: 0.809 - ETA: 1s - loss: 0.4623 - acc: 0.825 - ETA: 1s - loss: 0.4690 - acc: 0.823 - ETA: 0s - loss: 0.4657 - acc: 0.822 - ETA: 0s - loss: 0.4685 - acc: 0.819 - ETA: 0s - loss: 0.4798 - acc: 0.814 - 4s 9ms/step - loss: 0.4769 - acc: 0.8187 - val_loss: 0.5224 - val_acc: 0.8583

Epoch 00001: val_loss improved from inf to 0.52238, saving model to model.h5
Epoch 2/300
480/480 [==============================] - ETA: 1s - loss: 0.4461 - acc: 0.843 - ETA: 1s - loss: 0.3573 - acc: 0.859 - ETA: 1s - loss: 0.3759 - acc: 0.843 - ETA: 1s - loss: 0.4139 - acc: 0.843 - ETA

480/480 [==============================] - ETA: 0s - loss: 0.2497 - acc: 0.906 - ETA: 0s - loss: 0.3966 - acc: 0.875 - ETA: 0s - loss: 0.3559 - acc: 0.885 - ETA: 0s - loss: 0.3755 - acc: 0.867 - ETA: 0s - loss: 0.3815 - acc: 0.862 - ETA: 0s - loss: 0.4257 - acc: 0.849 - ETA: 0s - loss: 0.4232 - acc: 0.848 - ETA: 0s - loss: 0.4061 - acc: 0.851 - ETA: 0s - loss: 0.3937 - acc: 0.861 - ETA: 0s - loss: 0.3811 - acc: 0.865 - ETA: 0s - loss: 0.3824 - acc: 0.863 - ETA: 0s - loss: 0.3891 - acc: 0.854 - ETA: 0s - loss: 0.3959 - acc: 0.853 - ETA: 0s - loss: 0.4046 - acc: 0.850 - 1s 2ms/step - loss: 0.3961 - acc: 0.8562 - val_loss: 0.5647 - val_acc: 0.8417

Epoch 00012: val_loss did not improve from 0.45087
Epoch 13/300
480/480 [==============================] - ETA: 0s - loss: 0.2782 - acc: 0.906 - ETA: 0s - loss: 0.2896 - acc: 0.921 - ETA: 0s - loss: 0.3431 - acc: 0.885 - ETA: 0s - loss: 0.3078 - acc: 0.898 - ETA: 0s - loss: 0.2989 - acc: 0.900 - ETA: 0s - loss: 0.3271 - acc: 0.885 - ETA: 0s - l

480/480 [==============================] - ETA: 0s - loss: 0.2630 - acc: 0.843 - ETA: 0s - loss: 0.2500 - acc: 0.859 - ETA: 0s - loss: 0.2560 - acc: 0.875 - ETA: 0s - loss: 0.2369 - acc: 0.890 - ETA: 0s - loss: 0.2133 - acc: 0.906 - ETA: 0s - loss: 0.2258 - acc: 0.901 - ETA: 0s - loss: 0.2600 - acc: 0.897 - ETA: 0s - loss: 0.2621 - acc: 0.898 - ETA: 0s - loss: 0.2650 - acc: 0.899 - ETA: 0s - loss: 0.2520 - acc: 0.906 - ETA: 0s - loss: 0.2411 - acc: 0.911 - ETA: 0s - loss: 0.2393 - acc: 0.911 - ETA: 0s - loss: 0.2619 - acc: 0.903 - ETA: 0s - loss: 0.2717 - acc: 0.904 - 1s 2ms/step - loss: 0.2681 - acc: 0.9083 - val_loss: 0.5655 - val_acc: 0.8500

Epoch 00023: val_loss did not improve from 0.45087
Epoch 24/300
480/480 [==============================] - ETA: 0s - loss: 0.1435 - acc: 0.968 - ETA: 0s - loss: 0.1315 - acc: 0.953 - ETA: 0s - loss: 0.2331 - acc: 0.916 - ETA: 0s - loss: 0.2417 - acc: 0.906 - ETA: 0s - loss: 0.2884 - acc: 0.887 - ETA: 0s - loss: 0.2813 - acc: 0.895 - ETA: 0s - l

480/480 [==============================] - ETA: 1s - loss: 0.2132 - acc: 0.906 - ETA: 0s - loss: 0.2671 - acc: 0.875 - ETA: 0s - loss: 0.2517 - acc: 0.875 - ETA: 0s - loss: 0.2417 - acc: 0.882 - ETA: 0s - loss: 0.2044 - acc: 0.900 - ETA: 0s - loss: 0.2138 - acc: 0.901 - ETA: 0s - loss: 0.2156 - acc: 0.901 - ETA: 0s - loss: 0.2209 - acc: 0.898 - ETA: 0s - loss: 0.2257 - acc: 0.895 - ETA: 0s - loss: 0.2266 - acc: 0.903 - ETA: 0s - loss: 0.2171 - acc: 0.906 - ETA: 0s - loss: 0.2190 - acc: 0.903 - ETA: 0s - loss: 0.2252 - acc: 0.901 - ETA: 0s - loss: 0.2234 - acc: 0.904 - 1s 2ms/step - loss: 0.2246 - acc: 0.9062 - val_loss: 0.4778 - val_acc: 0.8667

Epoch 00034: val_loss did not improve from 0.45087
Epoch 35/300
480/480 [==============================] - ETA: 0s - loss: 0.1775 - acc: 0.937 - ETA: 0s - loss: 0.2499 - acc: 0.921 - ETA: 0s - loss: 0.2198 - acc: 0.927 - ETA: 0s - loss: 0.2116 - acc: 0.929 - ETA: 0s - loss: 0.2067 - acc: 0.937 - ETA: 0s - loss: 0.2147 - acc: 0.937 - ETA: 0s - l

480/480 [==============================] - ETA: 0s - loss: 0.1783 - acc: 0.937 - ETA: 0s - loss: 0.1594 - acc: 0.937 - ETA: 0s - loss: 0.1407 - acc: 0.947 - ETA: 0s - loss: 0.1390 - acc: 0.953 - ETA: 0s - loss: 0.1633 - acc: 0.937 - ETA: 0s - loss: 0.1569 - acc: 0.942 - ETA: 0s - loss: 0.1611 - acc: 0.942 - ETA: 0s - loss: 0.1700 - acc: 0.933 - ETA: 0s - loss: 0.1694 - acc: 0.934 - ETA: 0s - loss: 0.1644 - acc: 0.937 - ETA: 0s - loss: 0.1605 - acc: 0.943 - ETA: 0s - loss: 0.1568 - acc: 0.940 - ETA: 0s - loss: 0.1519 - acc: 0.942 - ETA: 0s - loss: 0.1466 - acc: 0.946 - 1s 2ms/step - loss: 0.1477 - acc: 0.9458 - val_loss: 0.4570 - val_acc: 0.8833

Epoch 00045: val_loss did not improve from 0.45011
Epoch 46/300
480/480 [==============================] - ETA: 0s - loss: 0.0788 - acc: 1.000 - ETA: 0s - loss: 0.0858 - acc: 0.968 - ETA: 0s - loss: 0.0835 - acc: 0.968 - ETA: 0s - loss: 0.0843 - acc: 0.976 - ETA: 0s - loss: 0.0928 - acc: 0.975 - ETA: 0s - loss: 0.1060 - acc: 0.968 - ETA: 0s - l

...

480/480 [==============================] - ETA: 0s - loss: 0.1230 - acc: 0.937 - ETA: 0s - loss: 0.0825 - acc: 0.968 - ETA: 0s - loss: 0.0722 - acc: 0.968 - ETA: 0s - loss: 0.0649 - acc: 0.968 - ETA: 0s - loss: 0.0606 - acc: 0.975 - ETA: 0s - loss: 0.0689 - acc: 0.974 - ETA: 0s - loss: 0.0661 - acc: 0.977 - ETA: 0s - loss: 0.0676 - acc: 0.976 - ETA: 0s - loss: 0.0717 - acc: 0.975 - ETA: 0s - loss: 0.0663 - acc: 0.978 - ETA: 0s - loss: 0.0739 - acc: 0.977 - ETA: 0s - loss: 0.0826 - acc: 0.968 - ETA: 0s - loss: 0.0832 - acc: 0.968 - ETA: 0s - loss: 0.0926 - acc: 0.964 - 1s 2ms/step - loss: 0.0895 - acc: 0.9667 - val_loss: 0.6760 - val_acc: 0.8500

Epoch 00254: val_loss did not improve from 0.43936
Epoch 255/300
480/480 [==============================] - ETA: 0s - loss: 0.1146 - acc: 0.968 - ETA: 0s - loss: 0.0933 - acc: 0.968 - ETA: 0s - loss: 0.0925 - acc: 0.979 - ETA: 0s - loss: 0.0858 - acc: 0.976 - ETA: 0s - loss: 0.0994 - acc: 0.968 - ETA: 0s - loss: 0.1024 - acc: 0.963 - ETA: 0s - 

480/480 [==============================] - ETA: 0s - loss: 0.0453 - acc: 1.000 - ETA: 0s - loss: 0.0604 - acc: 1.000 - ETA: 0s - loss: 0.0559 - acc: 0.989 - ETA: 0s - loss: 0.0519 - acc: 0.992 - ETA: 0s - loss: 0.0518 - acc: 0.993 - ETA: 0s - loss: 0.0525 - acc: 0.989 - ETA: 0s - loss: 0.0479 - acc: 0.991 - ETA: 0s - loss: 0.0434 - acc: 0.992 - ETA: 0s - loss: 0.0478 - acc: 0.989 - ETA: 0s - loss: 0.0504 - acc: 0.987 - ETA: 0s - loss: 0.0503 - acc: 0.988 - ETA: 0s - loss: 0.0475 - acc: 0.989 - ETA: 0s - loss: 0.0469 - acc: 0.990 - ETA: 0s - loss: 0.0442 - acc: 0.991 - 1s 2ms/step - loss: 0.0457 - acc: 0.9896 - val_loss: 0.8352 - val_acc: 0.8583

Epoch 00265: val_loss did not improve from 0.43936
Epoch 266/300
480/480 [==============================] - ETA: 0s - loss: 0.1157 - acc: 0.937 - ETA: 0s - loss: 0.0714 - acc: 0.968 - ETA: 0s - loss: 0.0814 - acc: 0.968 - ETA: 0s - loss: 0.0718 - acc: 0.968 - ETA: 0s - loss: 0.0745 - acc: 0.968 - ETA: 0s - loss: 0.0710 - acc: 0.968 - ETA: 0s - 

480/480 [==============================] - ETA: 0s - loss: 0.0377 - acc: 1.000 - ETA: 0s - loss: 0.0443 - acc: 0.984 - ETA: 0s - loss: 0.0327 - acc: 0.989 - ETA: 0s - loss: 0.0445 - acc: 0.992 - ETA: 0s - loss: 0.0397 - acc: 0.993 - ETA: 0s - loss: 0.0353 - acc: 0.994 - ETA: 0s - loss: 0.0353 - acc: 0.995 - ETA: 0s - loss: 0.0333 - acc: 0.996 - ETA: 0s - loss: 0.0407 - acc: 0.993 - ETA: 0s - loss: 0.0470 - acc: 0.987 - ETA: 0s - loss: 0.0452 - acc: 0.988 - ETA: 0s - loss: 0.0425 - acc: 0.989 - ETA: 0s - loss: 0.0408 - acc: 0.990 - ETA: 0s - loss: 0.0422 - acc: 0.991 - 1s 2ms/step - loss: 0.0510 - acc: 0.9875 - val_loss: 0.8982 - val_acc: 0.8750

Epoch 00276: val_loss did not improve from 0.43936
Epoch 277/300
480/480 [==============================] - ETA: 0s - loss: 0.0378 - acc: 1.000 - ETA: 0s - loss: 0.0597 - acc: 0.984 - ETA: 0s - loss: 0.0501 - acc: 0.989 - ETA: 0s - loss: 0.0450 - acc: 0.992 - ETA: 0s - loss: 0.0386 - acc: 0.993 - ETA: 0s - loss: 0.0395 - acc: 0.994 - ETA: 0s - 

480/480 [==============================] - ETA: 0s - loss: 0.0154 - acc: 1.000 - ETA: 0s - loss: 0.0489 - acc: 0.968 - ETA: 0s - loss: 0.0777 - acc: 0.947 - ETA: 0s - loss: 0.0594 - acc: 0.960 - ETA: 0s - loss: 0.0479 - acc: 0.968 - ETA: 0s - loss: 0.0408 - acc: 0.974 - ETA: 0s - loss: 0.0496 - acc: 0.973 - ETA: 0s - loss: 0.0478 - acc: 0.976 - ETA: 0s - loss: 0.0442 - acc: 0.979 - ETA: 0s - loss: 0.0459 - acc: 0.975 - ETA: 0s - loss: 0.0469 - acc: 0.977 - ETA: 0s - loss: 0.0466 - acc: 0.976 - ETA: 0s - loss: 0.0485 - acc: 0.976 - ETA: 0s - loss: 0.0482 - acc: 0.975 - 1s 2ms/step - loss: 0.0520 - acc: 0.9750 - val_loss: 0.9270 - val_acc: 0.8750

Epoch 00287: val_loss did not improve from 0.43936
Epoch 288/300
480/480 [==============================] - ETA: 0s - loss: 0.0301 - acc: 0.968 - ETA: 0s - loss: 0.0508 - acc: 0.968 - ETA: 0s - loss: 0.0989 - acc: 0.947 - ETA: 0s - loss: 0.1028 - acc: 0.937 - ETA: 0s - loss: 0.1096 - acc: 0.937 - ETA: 0s - loss: 0.0936 - acc: 0.947 - ETA: 0s - 

480/480 [==============================] - ETA: 0s - loss: 0.0980 - acc: 0.937 - ETA: 0s - loss: 0.0752 - acc: 0.953 - ETA: 0s - loss: 0.0771 - acc: 0.958 - ETA: 0s - loss: 0.2202 - acc: 0.953 - ETA: 0s - loss: 0.1860 - acc: 0.962 - ETA: 0s - loss: 0.1711 - acc: 0.963 - ETA: 0s - loss: 0.1563 - acc: 0.964 - ETA: 0s - loss: 0.1462 - acc: 0.964 - ETA: 0s - loss: 0.1483 - acc: 0.958 - ETA: 0s - loss: 0.1433 - acc: 0.959 - ETA: 0s - loss: 0.1336 - acc: 0.963 - ETA: 0s - loss: 0.1254 - acc: 0.963 - ETA: 0s - loss: 0.1210 - acc: 0.963 - ETA: 0s - loss: 0.1165 - acc: 0.964 - 1s 2ms/step - loss: 0.1117 - acc: 0.9667 - val_loss: 0.7137 - val_acc: 0.9167

Epoch 00298: val_loss did not improve from 0.43936
Epoch 299/300
480/480 [==============================] - ETA: 0s - loss: 0.0142 - acc: 1.000 - ETA: 0s - loss: 0.0580 - acc: 0.984 - ETA: 0s - loss: 0.0462 - acc: 0.989 - ETA: 0s - loss: 0.0535 - acc: 0.984 - ETA: 0s - loss: 0.0534 - acc: 0.981 - ETA: 0s - loss: 0.0487 - acc: 0.979 - ETA: 0s - 

In [43]:
 model = load_model("model.h5")

In [36]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png', show_shapes=True,show_layer_names=True)
# from IPython.display import display, Image
# display(Image(filename='model.png'))


In [35]:
import matplotlib.pyplot as plt

# # Plot training & validation accuracy values
# plt.plot(hist.history['acc'])
# plt.plot(hist.history['val_acc'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(hist.history['loss'])
# plt.plot(hist.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

In [36]:
def predictions(text):
#     clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
    test_word = viTokenList(text)
    test_word = [w.lower() for w in test_word]
    test_ls = word_tokenizer.texts_to_sequences(test_word)
#     print("test_ls: ", test_ls)
#     print("test_word: ", test_word)
    #Check for unknown words
    if [] in test_ls:
        test_ls = list(filter(None, test_ls))

    test_ls = np.array(test_ls).reshape(1, len(test_ls))

    x = padding_doc(test_ls, max_length)

    pred = model.predict_proba(x)


    return pred


  

In [37]:
def load_response_dataset(filename):
    df_res = pd.read_excel(filename, sheet_name="Tag_Response", encoding="utf8") 
    return df_res

In [38]:
import random
def response(classes):
    df_res = load_response_dataset("dataset_XY_XLS_updatedbytho.xls")
    s = df_res[classes].dropna()
    s_res = s[random.randint(0,len(s)-1)]
    print(s_res)

In [39]:
def get_final_output(pred, classes):
    predictions = pred[0]

    classes = np.array(classes)
    ids = np.argsort(-predictions)
    classes = classes[ids]
    predictions = -np.sort(-predictions)
 
    for i in range(3):
        print("%s has confidence = %s" % (classes[i], (predictions[i])))
    response(classes[0])
#     print(classes[0])

In [ ]:

while(1):
    text = input()    
    if text == 'q':
        break
    pred = predictions(text)
    get_final_output(pred, unique_intent)
    

hi
Chức_năng_của_bot has confidence = 0.7883284
Nơi_nộp_hồ_sơ has confidence = 0.14198619
Thời_gian_giải_quyết_hộ_chiếu has confidence = 0.039299227
Tôi là Chatbot hỗ trợ bạn trong việc tư vấn về thủ tục Hành Chính Công, cụ thể là về thủ tục passport. Có hai loại hộ chiếu là hộ chiếu cấp Trung ương và cấp tỉnh. Bạn nên lưu ý ghi rõ là cấp nào.
ngu luôn
Chức_năng_của_bot has confidence = 0.55176234
Thời_gian_giải_quyết_hộ_chiếu has confidence = 0.34406018
Tên_bot has confidence = 0.07534421
Tôi là Chatbot hỗ trợ bạn trong việc tư vấn về thủ tục Hành Chính Công, cụ thể là về thủ tục passport. Có hai loại hộ chiếu là hộ chiếu cấp Trung ương và cấp tỉnh. Bạn nên lưu ý ghi rõ là cấp nào.
sao m bị ngu rồi?
Chức_năng_của_bot has confidence = 0.54097444
Thời_gian_giải_quyết_hộ_chiếu has confidence = 0.3523135
Tên_bot has confidence = 0.07309757
Tôi là Chatbot hỗ trợ bạn trong việc tư vấn về thủ tục Hành Chính Công, cụ thể là về thủ tục passport. Có hai loại hộ chiếu là hộ chiếu cấp Trung ương 

In [44]:
print(model.metrics_names)
print(model.test_on_batch(batch_X, batch_Y, sample_weight=None))

['loss', 'acc']
[0.4393612, 0.89166665]
